In [9]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.callbacks import get_openai_callback
from langchain.chains.question_answering import load_qa_chain

from dotenv import load_dotenv
import os

In [3]:
text = """생성형 인공지능 또는 생성형 AI는 프롬프트에 대응하여 텍스트, 이미지, 기타 미디어를 생성할 수 있는 일종의 인공지능 시스템이다.
단순히 기존 데이터를 분석하는 것이 아닌, 새로운 콘텐츠를 만드는 데 초점을 맞춘 인공지능 분야를 말한다. 2022년경부터 본격적으로 유명해지기 시작했다.
데이터 원본을 통한 학습으로 소설, 이미지, 비디오, 코딩, 음악, 미술 등 다양한 콘텐츠 생성에 이용된다. 한국에서는 2022년 Novel AI 등, 그림 인공지능의 등장으로 주목도가 높아졌으며, 해외에서는 미드저니, 챗GPT등 여러 모델을 잇달아 공개하면서 화제의 중심이 되었다.
보통 딥러닝 인공지능은 학습 혹은 결과 출력 전 원본 자료를 배열 자료형[2] 숫자 데이터로 변환하는 인코딩 과정이 중요한데, 생성 AI의 경우 인공지능의 출력 데이터를 역으로 그림, 글 등의 원하는 형태로 변환시켜주는 디코딩 과정 또한 필요하다.
사실상 인공지능의 대중화를 이끈 기술로써, 해당 기술이 인공지능에 대한 사람들의 전반적인 인식을 매우 크게 바꿔놨다고 해도 과언이 아니다.
"""
#CharacterTextSplitter을 이용한 청킹
splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=300,
    chunk_overlap=50,
    length_function=len,
    )

#주어진 문장 청킹
chunks = splitter.split_text(text)
print(chunks)

['생성형 인공지능 또는 생성형 AI는 프롬프트에 대응하여 텍스트, 이미지, 기타 미디어를 생성할 수 있는 일종의 인공지능 시스템이다.\n단순히 기존 데이터를 분석하는 것이 아닌, 새로운 콘텐츠를 만드는 데 초점을 맞춘 인공지능 분야를 말한다. 2022년경부터 본격적으로 유명해지기 시작했다.', '데이터 원본을 통한 학습으로 소설, 이미지, 비디오, 코딩, 음악, 미술 등 다양한 콘텐츠 생성에 이용된다. 한국에서는 2022년 Novel AI 등, 그림 인공지능의 등장으로 주목도가 높아졌으며, 해외에서는 미드저니, 챗GPT등 여러 모델을 잇달아 공개하면서 화제의 중심이 되었다.\n보통 딥러닝 인공지능은 학습 혹은 결과 출력 전 원본 자료를 배열 자료형[2] 숫자 데이터로 변환하는 인코딩 과정이 중요한데, 생성 AI의 경우 인공지능의 출력 데이터를 역으로 그림, 글 등의 원하는 형태로 변환시켜주는 디코딩 과정 또한 필요하다.', '사실상 인공지능의 대중화를 이끈 기술로써, 해당 기술이 인공지능에 대한 사람들의 전반적인 인식을 매우 크게 바꿔놨다고 해도 과언이 아니다.']


In [16]:
len(chunks)

3

In [15]:
for sentence in chunks:
    print(sentence)

생성형 인공지능 또는 생성형 AI는 프롬프트에 대응하여 텍스트, 이미지, 기타 미디어를 생성할 수 있는 일종의 인공지능 시스템이다.
단순히 기존 데이터를 분석하는 것이 아닌, 새로운 콘텐츠를 만드는 데 초점을 맞춘 인공지능 분야를 말한다. 2022년경부터 본격적으로 유명해지기 시작했다.
데이터 원본을 통한 학습으로 소설, 이미지, 비디오, 코딩, 음악, 미술 등 다양한 콘텐츠 생성에 이용된다. 한국에서는 2022년 Novel AI 등, 그림 인공지능의 등장으로 주목도가 높아졌으며, 해외에서는 미드저니, 챗GPT등 여러 모델을 잇달아 공개하면서 화제의 중심이 되었다.
보통 딥러닝 인공지능은 학습 혹은 결과 출력 전 원본 자료를 배열 자료형[2] 숫자 데이터로 변환하는 인코딩 과정이 중요한데, 생성 AI의 경우 인공지능의 출력 데이터를 역으로 그림, 글 등의 원하는 형태로 변환시켜주는 디코딩 과정 또한 필요하다.
사실상 인공지능의 대중화를 이끈 기술로써, 해당 기술이 인공지능에 대한 사람들의 전반적인 인식을 매우 크게 바꿔놨다고 해도 과언이 아니다.


In [5]:
load_dotenv()

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002", 
    api_key=os.getenv("OPENAI_API_KEY"),
    )

#텍스트 임베딩을 FAISS FAISS(Vector Store)에 저장
knowledge_base = FAISS.from_texts(chunks, embeddings)

In [6]:
print(knowledge_base)

In [7]:
question = "생성형 AI란?"
references = knowledge_base.similarity_search(question)

In [8]:
references

[Document(id='add5d65d-665e-4aa4-94f3-c91fa74dfbc3', metadata={}, page_content='생성형 인공지능 또는 생성형 AI는 프롬프트에 대응하여 텍스트, 이미지, 기타 미디어를 생성할 수 있는 일종의 인공지능 시스템이다.\n단순히 기존 데이터를 분석하는 것이 아닌, 새로운 콘텐츠를 만드는 데 초점을 맞춘 인공지능 분야를 말한다. 2022년경부터 본격적으로 유명해지기 시작했다.'),
 Document(id='1dffc505-ce78-4265-bc6f-a38825892915', metadata={}, page_content='데이터 원본을 통한 학습으로 소설, 이미지, 비디오, 코딩, 음악, 미술 등 다양한 콘텐츠 생성에 이용된다. 한국에서는 2022년 Novel AI 등, 그림 인공지능의 등장으로 주목도가 높아졌으며, 해외에서는 미드저니, 챗GPT등 여러 모델을 잇달아 공개하면서 화제의 중심이 되었다.\n보통 딥러닝 인공지능은 학습 혹은 결과 출력 전 원본 자료를 배열 자료형[2] 숫자 데이터로 변환하는 인코딩 과정이 중요한데, 생성 AI의 경우 인공지능의 출력 데이터를 역으로 그림, 글 등의 원하는 형태로 변환시켜주는 디코딩 과정 또한 필요하다.'),
 Document(id='fff27b34-ad8f-493c-94b7-e76105970865', metadata={}, page_content='사실상 인공지능의 대중화를 이끈 기술로써, 해당 기술이 인공지능에 대한 사람들의 전반적인 인식을 매우 크게 바꿔놨다고 해도 과언이 아니다.')]

references는 FAISS 벡터 데이터베이스로부터 질문과 유사한 청크(텍스트)를 검색한 결과
- references는 질문과 가장 유사한 문서 조각들을 반환한 결과이며, 이걸 바탕으로 답변 생성, 요약, 출처 제공 등에 활용 가능

In [10]:
llm = ChatOpenAI(model="gpt-4o", 
                 api_key=os.getenv("OPENAI_API_KEY"),
                 )


chain = load_qa_chain(llm, chain_type="stuff") #Q&A Interface, 하지만 모든 document를 가져온다
with get_openai_callback() as cb:
    response = chain.run(input_documents=references, question=question)
    print(cb)

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_29972\3899588587.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o",
C:\Users\jjsd4\AppData\Local\Temp\ipykernel_29972\3899588587.py:6: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See a

Tokens Used: 439
	Prompt Tokens: 345
		Prompt Tokens Cached: 0
	Completion Tokens: 94
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.0018024999999999998


문서를 입력받아 Q&A를 수행하는 LangChain QA 체인을 생성
- chain_type="stuff"는 모든 문서를 하나로 '쑤셔 넣고(stuff)' 질문에 전달함
- 문서가 짧을 때는 좋지만, 길면 token limit 초과 가능성 있음
- 대안: "map_reduce", "refine", "map_rerank" 등

get_openai_callback(): 사용된 토큰 수, 비용 등을 추적

In [11]:
response

'생성형 AI는 프롬프트에 대응하여 텍스트, 이미지, 기타 미디어를 생성할 수 있는 인공지능 시스템입니다. 기존 데이터를 분석하는 것에 그치지 않고, 새로운 콘텐츠를 만드는 데 초점을 맞춘 AI 분야입니다. 소설, 이미지, 비디오, 코딩, 음악 등 다양한 콘텐츠 생성에 이용되며, 최근 몇 년 간 대중화되며 많은 관심을 받고 있는 기술입니다.'

In [13]:
for line in response.split(". "):
    print(line)


생성형 AI는 프롬프트에 대응하여 텍스트, 이미지, 기타 미디어를 생성할 수 있는 인공지능 시스템입니다
기존 데이터를 분석하는 것에 그치지 않고, 새로운 콘텐츠를 만드는 데 초점을 맞춘 AI 분야입니다
소설, 이미지, 비디오, 코딩, 음악 등 다양한 콘텐츠 생성에 이용되며, 최근 몇 년 간 대중화되며 많은 관심을 받고 있는 기술입니다.
